## Preprocessing and Training Data Development - Vacancy Rates 
### Goal:  Create a cleaned development dataset you can use to complete the modeling step of your project


#### Steps: 
● 1. Create dummy or indicator features for categorical variables

● 2. Standardize the magnitude of numeric features using a scaler

● 3. Split into testing and training datasets

In [27]:
#imports
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import __version__ as sklearn_version
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression
import datetime
from pandas_profiling import ProfileReport

In [28]:
#load data
path= '/Users/josephfrasca/Coding_Stuff/Springboard/Capstone_2/data/interim'
os.chdir(path) 

In [29]:
# load cleaned data
df = pd.read_csv('vacancy_zillowHomeRent_merge_2014_2020.csv', dtype={'Zipcode': object})
df

,Zipcode,RentPrice,Year,SizeRank,State,City,Metro,CountyName,HomePrice,Vacancy_Rate%,MOE-VacancyRate%
0,10025,3041.83,2014,0.0,NY,New York,New York-Newark-Jersey City,New York County,968761.75,9.011810,1.539867
1,60657,1589.42,2014,1.0,IL,Chicago,Chicago-Naperville-Elgin,Cook County,450755.75,8.042922,1.343734
2,10023,3186.67,2014,2.0,NY,New York,New York-Newark-Jersey City,New York County,1024543.17,19.964756,2.296513
3,77494,1807.33,2014,3.0,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,322032.00,3.319292,1.229599
4,60614,1786.25,2014,4.0,IL,Chicago,Chicago-Naperville-Elgin,Cook County,580250.92,8.468203,1.250484
...,...,...,...,...,...,...,...,...,...,...,...
22696,02110,4408.57,2020,14752.0,MA,Boston,Boston-Cambridge-Newton,Suffolk County,1339232.44,NaN,NaN
22697,20004,2505.56,2020,15149.0,DC,Washington,Washington-Arlington-Alexandria,District of Columbia,497022.00,NaN,NaN
22698,80951,1647.88,2020,15318.0,CO,Colorado Springs,Colorado Springs,El Paso County,315486.22,NaN,NaN
22699,11964,15800.50,2020,17169.0,NY,Town of Shelter Island,New York-Newark-Jersey City,Suffolk County,1015162.00,NaN,NaN


In [30]:
#drop margin of error of vacancy rate
df.drop('MOE-VacancyRate%', axis=1, inplace=True)

In [31]:
df.dtypes

Zipcode           object
RentPrice        float64
Year               int64
SizeRank         float64
State             object
City              object
Metro             object
CountyName        object
HomePrice        float64
Vacancy_Rate%    float64
dtype: object

In [32]:
#split into two dataframes for future modeling and predicting vacancy rates in 2019-2020
df_2019_2020 = df[df.Year > 2018]
df = df[df.Year < 2019]
#check NaNs
df.isna().sum()/len(df)*100

Zipcode          0.000000
RentPrice        0.018501
Year             0.000000
SizeRank         0.123343
State            0.092507
City             0.092507
Metro            0.092507
CountyName       0.092507
HomePrice        0.240518
Vacancy_Rate%    0.154178
dtype: float64

In [34]:
df_2019_2020.isna().sum()/len(df_2019_2020)*100

Zipcode            0.000000
RentPrice          0.000000
Year               0.000000
SizeRank           0.123343
State              0.092507
City               0.092507
Metro              0.092507
CountyName         0.092507
HomePrice          0.123343
Vacancy_Rate%    100.000000
dtype: float64

In [35]:
len(df), len(df_2019_2020)

(16215, 6486)

In [36]:
#drop NaNs
df.dropna(inplace=True)
df

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Zipcode,RentPrice,Year,SizeRank,State,City,Metro,CountyName,HomePrice,Vacancy_Rate%
0,10025,3041.83,2014,0.0,NY,New York,New York-Newark-Jersey City,New York County,968761.75,9.011810
1,60657,1589.42,2014,1.0,IL,Chicago,Chicago-Naperville-Elgin,Cook County,450755.75,8.042922
2,10023,3186.67,2014,2.0,NY,New York,New York-Newark-Jersey City,New York County,1024543.17,19.964756
3,77494,1807.33,2014,3.0,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,322032.00,3.319292
4,60614,1786.25,2014,4.0,IL,Chicago,Chicago-Naperville-Elgin,Cook County,580250.92,8.468203
...,...,...,...,...,...,...,...,...,...,...
16210,02110,4643.58,2018,14752.0,MA,Boston,Boston-Cambridge-Newton,Suffolk County,1363870.08,17.412045
16211,20004,2432.25,2018,15149.0,DC,Washington,Washington-Arlington-Alexandria,District of Columbia,480942.83,21.036585
16212,80951,1537.18,2018,15318.0,CO,Colorado Springs,Colorado Springs,El Paso County,276619.83,1.084746
16213,11964,20122.17,2018,17169.0,NY,Town of Shelter Island,New York-Newark-Jersey City,Suffolk County,1000069.25,62.044105


In [37]:
#check unique values for each column
df['CountyName'].value_counts()/len(df)*100

Los Angeles County    5.139955
Orange County         3.405995
Maricopa County       3.344067
Cook County           2.508051
San Diego County      2.043597
                        ...   
Kaufman County        0.030964
Platte County         0.030964
Cecil County          0.030964
Carver County         0.030964
Brazoria County       0.030964
Name: CountyName, Length: 269, dtype: float64

In [38]:
#check partition sizes with a 80/20 train/test split
print('train size:', len(df) * .8, 'test size:', len(df) * .2)

train size: 12918.400000000001 test size: 3229.6000000000004


###  1. Create dummy or indicator features for categorical variables
Hint: you’ll need to think about your old favorite pandas functions here like
get_dummies() . Consult this guide for help.
<https://towardsdatascience.com/the-dummys-guide-to-creating-dummy-variables-f21faddb1d40>

In [39]:
#change zipcode so it's not turned into a dummy variable
df.Zipcode = df.Zipcode.astype('int')
df.dtypes

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


Zipcode            int64
RentPrice        float64
Year               int64
SizeRank         float64
State             object
City              object
Metro             object
CountyName        object
HomePrice        float64
Vacancy_Rate%    float64
dtype: object

In [40]:
#get dummy variables for 'object' columns 
df = pd.get_dummies(df)

df

,Zipcode,RentPrice,Year,SizeRank,HomePrice,Vacancy_Rate%,State_AL,State_AR,State_AZ,State_CA,...,CountyName_Weber County,CountyName_Weld County,CountyName_Westchester County,CountyName_Will County,CountyName_Williamson County,CountyName_Wilson County,CountyName_Worcester County,CountyName_Yamhill County,CountyName_Yolo County,CountyName_York County
0,10025,3041.83,2014,0.0,968761.75,9.011810,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,60657,1589.42,2014,1.0,450755.75,8.042922,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10023,3186.67,2014,2.0,1024543.17,19.964756,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,77494,1807.33,2014,3.0,322032.00,3.319292,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,60614,1786.25,2014,4.0,580250.92,8.468203,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16210,2110,4643.58,2018,14752.0,1363870.08,17.412045,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16211,20004,2432.25,2018,15149.0,480942.83,21.036585,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16212,80951,1537.18,2018,15318.0,276619.83,1.084746,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16213,11964,20122.17,2018,17169.0,1000069.25,62.044105,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Split into testing and training datasets
Hint: don’t forget your sklearn functions here, like train_test_split().

In [41]:
#define variable X, y
X = df.drop('Vacancy_Rate%', axis=1)
y = df['Vacancy_Rate%']

In [42]:
#train test split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [43]:
#train, test split for timeseries
tss = TimeSeriesSplit(n_splits = 5)
for train_index, test_index in tss.split(X):
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index,:]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

### Establish Baseline Measurement Comparisons
Using a Dummy Regressor see what R2, MSE, and MAE would be if the mean of the DataFrames were used

In [15]:
#initial not even a model
train_mean = y_train.mean()

print(train_mean)

9.924510521172483


In [16]:
#Fit the dummy regressor on the training data
dumb_reg = DummyRegressor(strategy='mean')
dumb_reg.fit(X_train, y_train)
#create dummy regressor predictions 
y_tr_pred = dumb_reg.predict(X_train)
#Make prediction with the single value of the (training) mean.
y_te_pred = train_mean * np.ones(len(y_test))
r2_score(y_train, y_tr_pred), r2_score(y_test, y_te_pred)

(0.0, -0.0003615371793730393)

In [17]:
#establish baseline for mean absolute error and mean square error 
print('MAEs:', mean_absolute_error(y_train, y_tr_pred), mean_absolute_error(y_test, y_te_pred))
print('MSEs:', mean_squared_error(y_train, y_tr_pred), mean_squared_error(y_test, y_te_pred))

MAEs: 5.074338214794146 5.188601429791711
MSEs: 55.026135025439736 58.8483100007387


### 3. Standardize the magnitude of numeric features using a scaler
Hint: you might need to employ Python code like this:

In [18]:
#NOTE Decided not to use scaled data as it lead to negative values for R2 scores. 
#negative R2 may be because tried to scale data after creating dummy variables?

scaler = StandardScaler()
#fit the scaler on the training set
scaler.fit(X_train)
#apply the scaling to both the train and test split
X_tr_scaled = scaler.transform(X_train)
X_te_scaled = scaler.transform(X_test)

#### Initial Model: Train the model on the train split

In [19]:
lm = LinearRegression().fit(X_train, y_train)

In [20]:
#Make predictions using the model on both train and test splits
y_tr_pred = lm.predict(X_train)
y_te_pred = lm.predict(X_test)

In [21]:
#Assess model performance
# r^2 - train, test
r2 = r2_score(y_train, y_tr_pred), r2_score(y_test, y_te_pred)
print('r2:', r2)

r2: (0.7703801642196975, 0.743751199645821)


**This is markedly better performance than when using Dummy variable/mean for R^2 (see earlier):**

Dummy R2 = (0.0, -0.001031839268772039)

In [22]:
#MAE - train, test
mae = mean_absolute_error(y_train, y_tr_pred), mean_absolute_error(y_test, y_te_pred)
print('mae:', mae)

mae: (2.1220824261244076, 2.4379578940259976)


In [23]:
# MSE - train, test
mse = mean_squared_error(y_train, y_tr_pred), mean_squared_error(y_test, y_te_pred)
print('mse:', mse)

mse: (12.635092088166228, 15.07435889936279)


**This is markedly better performance than when using Dummy variable/mean for R^2 (see earlier):**

Dummy -

MAEs: 5.14329246126588 4.923647741748382

MSEs: 57.38168678632161 48.79685160338582

**MSE still high (possibly due to this being a large data set**

## Save processed data

In [44]:
#save vacancy rate data for modeling - remember to use random state=42!
df.to_csv(r'/Users/josephfrasca/Coding_Stuff/Springboard/Capstone_2/data/processed/VacancyRate_Zillow_2014_2018', index=False)
df_2019_2020.to_csv(r'/Users/josephfrasca/Coding_Stuff/Springboard/Capstone_2/data/processed/VacancyRate_Zillow_2019_2020', index=False)

In [25]:
#save the scaled training and test splits

#X_tr_scaled.to_csv(r'/Users/josephfrasca/Coding_Stuff/Springboard/Capstone_2/data/processed/X_tr_scaled', index=False)
#X_te_scaled

### Summary
This summary should provide a quick overview for someone wanting to know quickly why the given model was chosen for the next part of the business problem to help guide important business decisions.

- loaded vacancy rate/zillow home&rent price dataframe
- dropped margin of error (MOE) of vacancy rate, as we do not have this data for 2019-2020 and therefore would not be helpful with a predictive model
- split into two dataframes for future modeling (2014-2018) and also predicting vacancy rates in 2019-2020
- dropped NaNs, due to finding that there was less than .02% NaNs in each row
- inspected unique values and partition size with an 80/20 train/test split
- changed zipcode so it's not turned into a dummy variable
- created dummy variables for 'object' columns (ie. Metro, State, City and County)
-  Split data into testing and training datasets, y variable was vacancy rate
    - considered creating train/test based on time series and did after seeing R2 scores were similar and this was the correct method because we are dealing with Time Series data
    
- Established Baseline Measurement Comparisons
    - Used a Dummy Regressor see what R2, MSE, and MAE would be if the y-mean of 9.92% vacancy rate from training data was used

- NOTE Decided not to use scaled data as it led to negative values for R2 scores. Negative R2 may be because tried to scale data after creating dummy variables?

- Initial Model: Train the model on the train split using a linear regressor
- Made predictions using the model on both train and test splits
- Assessed model performance: r2: (train: 0.7703801642196975, test: 0.743751199645821)
    - This is markedly better performance than when using Dummy variable/mean for R^2 (see earlier): Dummy R2 = (train: 0.0, test: -0.0003615371793730393)
- Assessed MAE/MSE on train and test splits
    - mae: (train 2.1220824261244076, test 2.4379578940259976)
    - mse: (train 12.635092088166228, test 15.07435889936279)
    - This is markedly better performance than when using Dummy variable/mean(see earlier): 
        - Dummy - MAEs: train 5.074338214794146  test 5.188601429791711. 
        - Dummy MSEs: train 55.026135025439736 test 58.8483100007387
    - MSE still high (possibly due to this being a large data set
- Saved processed data